In [5]:
import requests
from web3 import Web3
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/bdfb19d0e7914c1c975dd89500550e06'))
tokenDict={'0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2':{'symbol':'WETH','decimal':18},
           '0x6b175474e89094c44da98b954eedeac495271d0f':{'symbol':'DAI','decimal':18},
           '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599':{'symbol':'WBTC','decimal':8},
           '0xdac17f958d2ee523a2206206994597c13d831ec7':{'symbol':'USDT','decimal':6},
           '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48':{'symbol':'USDC','decimal':6},
           '0xa47c8bf37f92abed4a126bda807a7b7498661acd':{'symbol':'UST','decimal':18},
          }
import re
def getFillPrice(txHash='0xaf8207e7a890b6a7ec1994f304680e93069b6e7e9aebe38c5722d7986265e845',sender='dead',rebatePct=0.75):
    txdetails=web3.eth.get_transaction(txHash)
    maxFeePerGas=txdetails['maxFeePerGas']
    maxPriorityFeePerGas=txdetails['maxPriorityFeePerGas']
    #transfer topic, shows incoming/outgoing tokens
    tradetopic='0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
    txreceipt=web3.eth.get_transaction_receipt(txHash)
    gasUsed=txreceipt['gasUsed']
    gasPrice=txreceipt['effectiveGasPrice']
    gasInETH=gasUsed*gasPrice/10**18
    txLogs=txreceipt['logs']
    blockBaseFee=gasPrice-maxPriorityFeePerGas
    rebateInETH=blockBaseFee*gasUsed/10**18*rebatePct
    
    uniqueFills = []
    for log in txLogs:
        tokenAddress=log['address'].lower()
        if(tokenAddress in tokenDict):
            if(web3.toHex(log['topics'][0])==tradetopic):
                #we are receiver
                symbol=tokenDict[tokenAddress]['symbol']
                decimal=tokenDict[tokenAddress]['decimal']
                buysell='Buy' if(re.search(sender,web3.toHex(log['topics'][2]).lower())) else 'Sell'
                uniqueFills.append({'address': tokenAddress, 'qty': web3.toInt(hexstr=log['data']),'symbol': symbol,'decimal': decimal,'direction': buysell})

    if(uniqueFills[0]['address']=='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'):
        legAfill=uniqueFills[0]['qty']/10**uniqueFills[0]['decimal']
        legAsymbol=uniqueFills[0]['symbol']
        
        legBfill=uniqueFills[1]['qty']/10**uniqueFills[1]['decimal']
        legBsymbol=uniqueFills[1]['symbol']
        
        direction=uniqueFills[0]['direction']
    else:
        legAfill=uniqueFills[1]['qty']/10**uniqueFills[1]['decimal']
        legAsymbol=uniqueFills[1]['symbol']
        
        legBfill=uniqueFills[0]['qty']/10**uniqueFills[0]['decimal']
        legBsymbol=uniqueFills[0]['symbol']
        
        direction=uniqueFills[1]['direction']
    
    if(direction=='Buy'):
        print("Bought",legAfill,legAsymbol,"for",legBfill,legBsymbol)
        print("TX cost:",gasInETH,"ETH")
        print("Rebate:",rebateInETH,"ETH")
        avgFill=legBfill/(legAfill-gasInETH+rebateInETH)
        print("Avg Fill Price",avgFill)
    else:
        print("Sold",legAfill,legAsymbol,"for",legBfill,legBsymbol)
        print("TX cost:",gasInETH,"ETH")
        print("Rebate:",rebateInETH,"ETH")
        avgFill=legBfill/(legAfill+gasInETH-rebateInETH)
        print("Avg Fill Price",avgFill)

In [6]:
getFillPrice('0xa037d93fa0fd5c267fe0d9c2e91145a672a6e82c9b50ae50fb957302dbd282d3',rebatePct=0.75)

Sold 200.0 WETH for 503030.086889 USDC
TX cost: 0.16314851969641975 ETH
Rebate: 0.023743824522314806 ETH
Avg Fill Price 2513.398536660729
